In [1]:
!pip install tensorflow keras scikit-learn nltk pandas numpy

In [2]:
import pandas as pd
import numpy as np
import nltk
import re
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords

# Download stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
# Load dataset
df = pd.read_csv("text_emotion.csv")

# Select relevant columns for each task
df = df[['content', 'sentiment']]
df.head()

,content,sentiment
0,@tiffanylue i know i was listenin to bad habi...,empty
1,Layin n bed with a headache ughhhh...waitin o...,sadness
2,Funeral ceremony...gloomy friday...,sadness
3,wants to hang out with friends SOON!,enthusiasm
4,@dannycastillo We want to trade with someone w...,neutral


In [5]:
# Function to clean text
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)  # Remove URLs
    text = re.sub(r'\@\w+|\#', '', text)  # Remove mentions and hashtags
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuations
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = " ".join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

df['content'] = df['content'].apply(clean_text)

# Encode sentiment labels
sentiment_label_encoder = LabelEncoder()
df['sentiment_label'] = sentiment_label_encoder.fit_transform(df['sentiment'])

In [6]:
# Save label encoders for deployment
with open("sentiment_label_encoder.pkl", "wb") as f:
    pickle.dump(sentiment_label_encoder, f)

In [7]:
# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['content'])

In [8]:
# Save tokenizer for deployment
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

In [9]:
# Convert text to sequences and pad them
sequences = tokenizer.texts_to_sequences(df['content'])
max_length = 50  # Max sequence length
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post", truncating="post")

In [10]:
# Splitting data for Sentiment Model
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(
    padded_sequences, df['sentiment_label'], test_size=0.2, random_state=42, stratify=df['sentiment_label']
)

# Convert labels to categorical
num_classes_s = len(sentiment_label_encoder.classes_)
y_train_s = tf.keras.utils.to_categorical(y_train_s, num_classes_s)
y_test_s = tf.keras.utils.to_categorical(y_test_s, num_classes_s)

In [11]:
# Function to create LSTM model
def create_lstm_model(output_classes):
    model = Sequential([
        Embedding(input_dim=10000, output_dim=128, input_length=max_length),
        LSTM(128, return_sequences=True),
        Dropout(0.2),
        LSTM(64),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(output_classes, activation='softmax')  # Multi-class classification
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [12]:
# Train Sentiment Analysis Model
sentiment_model = create_lstm_model(num_classes_s)
sentiment_model.summary()
sentiment_model.fit(X_train_s, y_train_s, epochs=10, batch_size=32, validation_data=(X_test_s, y_test_s))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.2079 - loss: 2.1837 - val_accuracy: 0.2160 - val_loss: 2.1476
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - accuracy: 0.2117 - loss: 2.1498 - val_accuracy: 0.2160 - val_loss: 2.1454
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - accuracy: 0.2108 - loss: 2.1530 - val_accuracy: 0.2160 - val_loss: 2.1469
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.2182 - loss: 2.1500 - val_accuracy: 0.2160 - val_loss: 2.1463
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 0.2214 - loss: 2.1414 - val_accuracy: 0.2676 - val_loss: 2.0353
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.2776 - loss: 2.0068 - val_accuracy: 0.3133 - val_loss: 1.9710
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.3453 - loss: 1.8861 - val_accuracy: 0.3184 - val_loss: 1.9672
Epoch 8/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.3857 -

In [14]:
# Save Model
sentiment_model.save("sentiment_lstm_model.h5")
print("Model, tokenizer, and label encoder saved successfully!")

Model, tokenizer, and label encoder saved successfully!
